In [ ]:
# Importamos bibliotecas.
import numpy as np
import matplotlib.pyplot as plt
import time

from scipy.integrate import simpson

In [ ]:
# Definimos el sistema diferencial.
def der(w, r, m, s, f, g):    
    ene = 1 - (2*m)/(r)
    rho = ( (2*w)/(r**2 * s) )*(f**2 + g**2)
    srr = (2*ene/r**2)*( w*(f**2 + g**2)/(ene*s) - (2*f*g)/(r*np.sqrt(ene)) - (f**2 - g**2)/np.sqrt(ene) )
    zet = (r/ene)*(rho - srr) - (2*m)/(r**2 * ene)
    
    dm = r**2 * rho
    ds = (s*r/ene)*(rho + srr)
    df = f*( zet/2 + 1/(r*np.sqrt(ene)) ) - g*(1/np.sqrt(ene) + w/(ene*s))
    dg = g*( zet/2 - 1/(r*np.sqrt(ene)) ) - f*(1/np.sqrt(ene) - w/(ene*s))
    
    return [dm, ds, df, dg]

In [ ]:
# Definimos el método RK4 en sí.
def rk4(w, r, m, s, f, g, h):
    k1 = der(w, r, m, s, f, g)
    k1m = k1[0]
    k1s = k1[1]
    k1f = k1[2]
    k1g = k1[3]

    k2 = der(w, r + 0.5*h, m + 0.5*k1m*h, s + 0.5*k1s*h, f + 0.5*k1f*h, g + 0.5*k1g*h)
    k2m = k2[0]
    k2s = k2[1]
    k2f = k2[2]
    k2g = k2[3]

    k3 = der(w, r + 0.5*h, m + 0.5*k2m*h, s + 0.5*k2s*h, f + 0.5*k2f*h, g + 0.5*k2g*h)
    k3m = k3[0]
    k3s = k3[1]
    k3f = k3[2]
    k3g = k3[3]

    k4 = der(w, r + h, m + k3m*h, s + k3s*h, f + k3f*h, g + k3g*h)
    k4m = k4[0]
    k4s = k4[1]
    k4f = k4[2]
    k4g = k4[3]

    rrk4 = r + h
    mrk4 = m + (k1m + 2*k2m + 2*k3m + k4m)*(h/6)
    srk4 = s + (k1s + 2*k2s + 2*k3s + k4s)*(h/6)
    frk4 = f + (k1f + 2*k2f + 2*k3f + k4f)*(h/6)
    grk4 = g + (k1g + 2*k2g + 2*k3g + k4g)*(h/6)
    
    return [rrk4, mrk4, srk4, frk4, grk4]

In [ ]:
# Definimos una función que solo se encargue de solucionar.
def solucionador(h, w, f1, rf):
    # Definimos las condiciones iniciales.
    r0 = h   # Posición inicial.
    n  = int( (rf - r0)/h )
    
    m0 = (2/3)*(f1**2)*(r0**3)*w           # Condición inicial para m(r).
    s0 = 1 + (1/3)*(f1**2)*(4*w - 1)*r0**2 # Condición inicial para \sigma(r).
    f0 = f1*r0                             # Condición inicial para f(r).
    g0 = (1/3)*f1*(w - 1)*(r0**2)          # Condición inicial para g(r).

    #--------------------------------------------
    # Comenzamos a solucionar.
    # Definimos arreglos para las soluciones.
    r = np.zeros(n)
    m = np.zeros(n)
    s = np.zeros(n)
    f = np.zeros(n)
    g = np.zeros(n)

    # Definimos las simetrías.
    r[0] = -r0
    m[0] = m0
    s[0] = s0
    f[0] = -f0
    g[0] = g0
    
    # Metemos las condiciones iniciales.
    r[1] = r0
    m[1] = m0
    s[1] = s0
    f[1] = f0
    g[1] = g0

    # Llenamos los arreglos.
    for i in range(1, len(r)-1):
        soluto = rk4(w, r[i], m[i], s[i], f[i], g[i], h)
        
        r[i+1] = soluto[0]
        m[i+1] = soluto[1]
        s[i+1] = soluto[2]
        f[i+1] = soluto[3]
        g[i+1] = soluto[4]
    
    return [w, r, m, s, f, g]

In [ ]:
def shooting(h, w, f1, rf):
    # Definimos la primer diferencial de 'w'.
    tolerancia = h**2
    dw = 0.0000000001

    # Definimos un contador para ver cuántas veces necesitamos solucionar.
    counter = 0

    # Solucionamos por primera vez.
    f = solucionador(h, w, f1, rf)[4]
    counter += 1

    while abs( f[-1] ) > tolerancia:

        # '>' para 2do excitado.
        if f[-1] > tolerancia:
            # Actualizamos la 'w'.
            w = w + dw

            # Volvemos a solucionar.
            f = solucionador(h, w, f1, rf)[4]

        else:
            # Aquí la 'w' ha sido muy grande, entonces debemos regresar.
            w = w - dw

            # Ahora hacemos más pequeña la varaición y volvemos a avanzar.
            dw = dw/2
            w = w + dw

            # Volvemos a solucionar.
            f = solucionador(h, w, f1, rf)[4]

        counter += 1

    # Solucionamos una última vez (pero no la contamos).
    solucion = solucionador(h, w, f1, rf)
    r = solucion[1]
    m = solucion[2]
    s = solucion[3]
    f = solucion[4]
    g = solucion[5]

    print(f'El shooting se realizó {counter} veces')

    return [w, r, m, s, f, g]

In [ ]:
# Construímos un graficador.
def graficador(h, rf, r, m, s, f, g):
    fig = plt.figure(figsize=(15, 10))
    fig.tight_layout()
    #fig.suptitle(f'Soluciones con $h = ${h}')
    
    # r vs m.
    ax = plt.subplot(2,2,1)
    ax.plot(r, m, color = 'maroon')
    ax.set_xlabel(r'$r$')
    ax.set_ylabel(r'$m$')
    ax.grid(True)
    ax.set_title(r'$ m $')
    
    # r vs s.
    ax = plt.subplot(2,2,2)
    ax.plot(r, s, color = 'orangered')
    ax.set_xlabel(r'$r$')
    ax.set_ylabel(r'$\sigma$')
    ax.grid(True)
    ax.set_title(r'$ \sigma $')
    
    # r vs f.
    ax = plt.subplot(2,2,3)
    ax.plot(r, f, color = 'blue')
    ax.set_xlabel(r'$r$')
    ax.set_ylabel(r'$f$')
    ax.grid(True)
    ax.set_title(r'$ f $')
    
    # r vs g.
    ax = plt.subplot(2,2,4)
    ax.plot(r, g, color = 'red')
    ax.set_xlabel(r'$r$')
    ax.set_ylabel(r'$g$')
    ax.grid(True)
    ax.set_title(r'$ g $')
    
    plt.show()

In [ ]:
# Encontramos las variables físicas importantes.
def variables(w, r, m, s, f, g):
    # Regresamos una adimensión.
    s00 = 1/s[-1]
    wt = w*s00
    com = m[-1]/r[-1]
    
    # Calculamos la carga conservada.
    r = np.array(r)
    f = np.array(f)
    g = np.array(g)
    j = f*f + g*g

    Q = simpson(j, r)
    
    # Calculamos la energía de amarre.
    mamarre = m[-1] - 2*Q

    # Imprimimos las variables importantes.
    print(f'{f1} \t {w} \t {rf} \t {wt} \t {m[-1]} \t {com} \t {Q} \t {mamarre}')

    return

In [ ]:
# Definimos la función principal.
def main(selector, f1, w, rf, h):

    # Llamamos al solucionador.
    if selector == 1:
        soluto = solucionador(h, w, f1, rf)
    else:
        soluto = shooting(h, w, f1, rf)
        
    w = soluto[0]
    r = soluto[1]
    m = soluto[2]
    s = soluto[3]
    f = soluto[4]
    g = soluto[5]

    # Calculamos las variables físicas que nos interesan.
    variables(w, r, m, s, f, g)

    # Graficamos.
    graficador(h, rf, r, m, s, f, g)  

    return

In [ ]:
# Ejecutamos.

# Definimos la condición de ejecución.
if __name__ == '__main__':
    
    # Definimos los parámetros de entrada.
    selector = 2 # 1 sin shooting, otra cosa con shooting.
    f1 = 0.028
    w  = 1.039365040025879
    rf = 180
    h  = 0.01
    
    main(selector, f1, w, rf, h)